In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from sklearn.model_selection import ParameterGrid
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold


In [ ]:
plt.rcParams['figure.facecolor'] = 'white'
sns.set(rc={'axes.facecolor':'white', 'figure.facecolor':'white'})
sns.set(rc={'ytick.labelcolor':'black','xtick.labelcolor':'black'}) 

<h1> Functions <h1>

In [ ]:
def train_test_errors (X, y, trees_grid, model):
    train_results = []
    test_results = []
    list_nb_trees = trees_grid

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

    for trees in list_nb_trees:
        model = model
        model.fit(X_train, y_train)

        train_results.append(mean_absolute_error(y_train, model.predict(X_train)))
        test_results.append(mean_absolute_error(y_test, model.predict(X_test)))
    
    plot_df = pd.DataFrame({'trees' : trees_grid, 'train_absolute_error': train_results, 'test_absolute_error': test_results})
    return plot_df

In [ ]:
def perform_gridsearch (model, param_grid, X, y, scoring):

    
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, scoring=scoring, verbose=2, return_train_score=True)
    
    grid_search.fit(X, y)
    results_df = pd.DataFrame(grid_search.cv_results_)

    return results_df, grid_search.best_estimator_


In [ ]:
def plot_grid_search_results(results_df, x_axis_column, hue):

    for i, col in enumerate(results_df.columns):
        if (results_df[col].dtype == 'int' or results_df[col].dtype == 'float64') and col not in [x_axis_column, hue, 'rank_test_score']:
            plt.figure(i)
            sns.lineplot(data=results_df, y=col, x=x_axis_column, hue=hue, ci=None)

<h1> EDA <h1>

In [ ]:
df = pd.read_csv('data/data.csv')

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
for col in df.columns:
    print('Unique values in column %s' %col,'\n', df[col].unique(),
        '\nNumber of unique values in column %s :' %col, df[col].nunique())

<h3> Doing one-hot encoding <h3>

In [ ]:
dummies_sex = pd.get_dummies(df['sex'], prefix='sex')
dummies_smoker = pd.get_dummies(df['smoker'], prefix='smoker')
dummies_region = pd.get_dummies(df['region'], prefix='region')

In [ ]:
cols = ['sex', 'smoker', 'region']
dfs = [df, dummies_sex, dummies_smoker, dummies_region]
df_with_dummies = pd.concat(dfs, axis = 1)
df_with_dummies = df_with_dummies.drop(columns = cols)

<h2> Visualizations <h2>

In [ ]:
for i, col in enumerate(df.columns):
    plt.figure(i)
    sns.histplot(x = col, data = df)

In [ ]:
for i, col in enumerate(df.columns):
    if df[col].dtype == 'int64' or df[col].dtype == 'float64':
        plt.figure(i)
        sns.boxplot(x = col, data = df)

In [ ]:
sns.pairplot(df)

In [ ]:
for i, col in enumerate(df.columns):
    if df[col].dtype == 'int64' or df[col].dtype == 'float64':
        plt.figure(i)
        sns.violinplot(x = col, data = df)

In [ ]:
information_by_age = df_with_dummies.groupby('age').agg(
    median_charges = ('charges', np.median), 
    mean_bmi = ('bmi', np.mean)
    ).reset_index()

In [ ]:
percent_of_smokers_age = (df_with_dummies.groupby('age')['smoker_yes'].sum() / (df_with_dummies.groupby('age')['smoker_no'].sum() + df_with_dummies.groupby('age')['smoker_yes'].sum())).reset_index()
percent_of_smokers_age = percent_of_smokers_age.drop(columns = 'age')
information_by_age['percent_of_smokers'] = round(percent_of_smokers_age, 2) * 100

In [ ]:
information_by_age.head()

In [ ]:
sns.scatterplot(x = 'age', y = 'median_charges', data = information_by_age)

In [ ]:
#We have one age group which stands out in case of median of charges. On the plot below we can see that this age group also has the largest proportion of smokers
information_by_age[information_by_age['median_charges'] == information_by_age['median_charges'].max()]

In [ ]:
sns.scatterplot(x = 'age', y = 'mean_bmi', data = information_by_age)

In [ ]:
sns.scatterplot(x = 'age', y = 'percent_of_smokers', data = information_by_age)

In [ ]:
information_by_region = df.groupby('region').agg(
    median_charges = ('charges', np.median), 
    mean_bmi = ('bmi', np.mean),
    mean_children = ('children', np.mean)
    ).reset_index()

In [ ]:
sns.barplot(x = 'region', y = 'median_charges', data = information_by_region)

In [ ]:
sns.barplot(x = 'region', y = 'mean_bmi', data = information_by_region)

In [ ]:
sns.barplot(x = 'region', y = 'mean_children', data = information_by_region)

In [ ]:
df.groupby('region')['region'].count()

In [ ]:
df.head()

In [ ]:
df.groupby(['sex', 'smoker'])['age'].count()

In [ ]:
df.groupby(['region', 'smoker'])['age'].count()

In [ ]:
df.groupby(['sex']).agg(
    mean_charges = ('charges', np.mean)).reset_index()

In [ ]:
corr = df_with_dummies.corr()
plt.rcParams['figure.figsize'] = [10, 8]
sns.heatmap(corr, xticklabels=corr.columns, yticklabels=corr.columns, annot=True, cmap=sns.diverging_palette(220, 20, as_cmap=True))

<h2>  Thoughts after performing EDA <h2>
 Our dataset is from Kaggle so as expected it is really clean and 'pretty'. <br> 
    
   1. There are many more younger patients in our dataset <br>

   2. We can observe a linear relationship between age and [mean bmi, mean charges]. Older people probably have more advanced illnesses and that's why the treatment is more expensive <br>

   3. There are no serious outliers in our dataset, I think that we can call more extreme observations from our dataset "natural outliers", because in real life we also observe situations where a single person spends a lot more on treatment or <br>
    or has larger BMI. There is no human error in the dataset. <br>

   4. One age group (age = 43) stands out when it comes to the median of charges. This age group also has the largest proportion of smokers <br>
    
   5. When we analyze the information grouped by region, we can see slight differences (people from northeast pay more, people from southeast have bigger BMI) <br>

   6. On average, men pay more for treatment

   7. There is a very strong correlation between being a smoker and charges






<h1> Modeling data <h1>

In [ ]:
df_with_dummies.head()

In [ ]:
#I am not splititng the data on train-validation-test because later I will perform GridSearchCV to choose the best version of the model, right now I want to run basic configurations and see first results
X = df_with_dummies.loc[:, df_with_dummies.columns != 'charges']
y = df_with_dummies['charges']

In [ ]:
#base model
trees_grid = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 120, 140, 160, 180, 200]
results_df = train_test_errors(X, y, trees_grid, RandomForestRegressor() )

In [ ]:
plot_df = results_df.melt(id_vars = 'trees', value_vars = ['train_absolute_error', 'test_absolute_error'])
sns.lineplot(data = plot_df, x = 'trees', y = 'value', hue = 'variable')

<h2> Hyperparameters tuning <h2>

In [ ]:
#Splitting our data to have 10% of data left for the final test of models
X_90, X_10, y_90, y_10 = train_test_split(X, y, shuffle=False, test_size=0.1)

In [ ]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(20, 400, num = 11)]

# Number of features to consider at every split
max_features = ['sqrt']

# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 50, num = 5)]
max_depth.append(None)

# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]

# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]

# Method of selecting samples for training each tree
bootstrap = [True, False]


In [ ]:
param_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [ ]:
#best estimator : RandomForestRegressor(bootstrap=False, max_depth=20, max_features='sqrt',n_estimators=210)
results_df, best_estimator = perform_gridsearch(RandomForestRegressor(), param_grid, X_90, y_90, 'neg_mean_absolute_error')

In [ ]:
#Let's look more into details of our gridsearch
pd.set_option('display.max_columns', None)
results = pd.read_csv('data/hyperparameters_tuning_rf.csv', index_col = [0])
results.head()

In [ ]:
results['param_n_estimators'].unique()

In [ ]:
for col in results.columns:
    if results[col].dtype == 'int' or results[col].dtype == 'float64':
        results[col] = abs(results[col])

In [ ]:
plot_grid_search_results(results, 'param_min_samples_leaf', 'param_n_estimators')

In [ ]:
plot_grid_search_results(results, 'param_max_depth', 'param_n_estimators')

After performing grid search, we may see expected observations on random forest performance:
1. We don't observe a situation of overfitting when the number of estimators is higher, but for example when n_estimators = 80, too much depth of a single tree leads to overfitting (test error starts to increase, where train error decreases)
2. Increasing max_depth of a tree significantly reduces the error, especially at the first steps (for example increasing the depth from 10 to 20)
3. Increasing min_samples_leaf and min_samples_split increases the error, because the tree has less opportunity to split and that's why it does not fit well to certain cases
4. Increasing the number of estimators on average helps the algorithm to make better decisions